In [8]:
#导入包
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

In [9]:
BATCH_SIZE=512#参加训练的批数
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)#加载minst的训练集，并且进行标准化
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)#加载minst的测试集，并且进行标准化


In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 第一个卷积进来的通道数为1,输出的通道数为6,卷积核的大小是5*5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)# 第二个卷积进来的通道数为6,输出的通道数为16,卷积核的大小是5*5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # 一个全连接层，它有16x4x4个输入特征，输出120个特征。
        self.fc2 = nn.Linear(120, 84)#120个输入特征，输出84个特征。
        self.fc3 = nn.Linear(84, 10)#84个输入特征，输出10个特征，对应于10个类别的输出

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))#对输入张量x进行卷积操作，然后使用ReLU激活函数进行非线性变换，最后进行最大池化操作。这个操作的作用是提取输入张量中的特征，并减小特征图的大小。
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)#对上一步得到的特征图进行卷积、ReLU和最大池化操作。这个操作的作用是进一步提取特征，并减小特征图的大小
        x = torch.flatten(x, 1) # 将上一步得到的特征图展平成一个向量。这个操作的作用是将特征图转换为全连接层的输入
        x = F.relu(self.fc1(x))#：对全连接层进行线性变换，并使用ReLU激活函数进行非线性变换。
        x = F.relu(self.fc2(x))#：对全连接层进行线性变换，并使用ReLU激活函数进行非线性变换。
        x = self.fc3(x)#对上一步得到的特征进行线性变换，得到最终的输出
        x = F.log_softmax(x, dim=1) # 对输出进行log(softmax)操作，得到最终的预测结果。
        return x

In [13]:
#定义超参数
EPOCHS=20 # 总共训练批次
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU
learning_rate = 0.01#定义学习率
#声明一个LeNet-5模型
model = Net()
# 判断是否可以使用GPU加速
if torch.cuda.is_available():
    model.cuda()
#采用Adam优化器
optimizer = optim.Adam(model.parameters(),lr=learning_rate)


#开始训练
for epoch in range(1, EPOCHS + 1):
    # 训练
    model.train()#将模型设置为训练模式。
    for batch_idx, (data, target) in enumerate(train_loader):#对于训练数据集中的每个批次，执行以下操作。
        data, target = data.to(device), target.to(device)#将数据和目标张量移动到指定的设备上
        optimizer.zero_grad()#将优化器的梯度缓存清零。
        output = model(data)#使用模型对数据进行前向传播，得到输出张量
        loss = F.nll_loss(output, target)#计算输出张量和目标张量之间的负对数似然损失。
        loss.backward()#计算损失相对于模型参数的梯度。
        optimizer.step()#使用优化器更新模型参数
        if(batch_idx+1)%30 == 0:#每隔30个批次，执行以下操作。
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            #打印训练轮次、批次号、总批次数、完成百分比和损失值。
    # 测试
    model.eval()
    test_loss = 0#初始化测试损失为0。
    correct = 0#初始化正确预测数为0。
    with torch.no_grad():#在评估模式下，不需要计算梯度。
        for data, target in test_loader:#对于测试数据集中的每个批次，执行以下操作。
            data, target = data.to(device), target.to(device)
            output = model(data)#这个语句使用模型对数据进行前向传递，并将输出存储在变量output中。
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()#计算预测值与目标标签之间的匹配情况，并将正确的预测数量添加到correct变量中。

    test_loss /= len(test_loader.dataset)#计算测试集的平均损失。
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 1 [1856/60000 (3%)]	Loss: 0.646860
Train Epoch: 1 [3776/60000 (6%)]	Loss: 0.422914
Train Epoch: 1 [5696/60000 (9%)]	Loss: 0.105898
Train Epoch: 1 [7616/60000 (13%)]	Loss: 0.247754
Train Epoch: 1 [9536/60000 (16%)]	Loss: 0.132188
Train Epoch: 1 [11456/60000 (19%)]	Loss: 0.076193
Train Epoch: 1 [13376/60000 (22%)]	Loss: 0.136784
Train Epoch: 1 [15296/60000 (25%)]	Loss: 0.099006
Train Epoch: 1 [17216/60000 (29%)]	Loss: 0.210394
Train Epoch: 1 [19136/60000 (32%)]	Loss: 0.277062
Train Epoch: 1 [21056/60000 (35%)]	Loss: 0.204894
Train Epoch: 1 [22976/60000 (38%)]	Loss: 0.094259
Train Epoch: 1 [24896/60000 (41%)]	Loss: 0.285701
Train Epoch: 1 [26816/60000 (45%)]	Loss: 0.127354
Train Epoch: 1 [28736/60000 (48%)]	Loss: 0.131698
Train Epoch: 1 [30656/60000 (51%)]	Loss: 0.185212
Train Epoch: 1 [32576/60000 (54%)]	Loss: 0.299062
Train Epoch: 1 [34496/60000 (57%)]	Loss: 0.142639
Train Epoch: 1 [36416/60000 (61%)]	Loss: 0.064158
Train Epoch: 1 [38336/60000 (64%)]	Loss: 0.132928
Train Ep